In [62]:
import crypten
import torch

crypten.init()
from crypten.config import cfg

In [63]:
from crypten.config import cfg


In [64]:
# cfg.set_config(reciprocal_nr_iters= 100)

In [65]:
def chol_LD_Crypten(crypt_PSD_mat):

    L_enc = crypten.cryptensor(torch.zeros(crypt_PSD_mat.shape))
    D_enc = crypten.cryptensor(torch.zeros(crypt_PSD_mat.shape[0]))

    for i in (range(crypt_PSD_mat.shape[0])):
        L_enc[i,i] = 1 
        for j in range(crypt_PSD_mat.shape[0]):
            if i == j:
                if i == 0:
                    D_enc[i] = crypt_PSD_mat[i,i]
                else:
                    D_enc[i] = (crypt_PSD_mat[j,j] - L_enc[j,:j].t() @ (L_enc[j,:j]*D_enc[:j]))
            if i > j:
                    if j>0:
                        L_enc[i,j] = (crypt_PSD_mat[i,j] - L_enc[i,:j].t()@ (L_enc[j,:j]*D_enc[:j])).div(D_enc[j])
                    else:
                        L_enc[i,j] = (crypt_PSD_mat[i,j]).div(D_enc[j])
    
    return L_enc, D_enc

def logdet_Crypten(crypt_PSD_mat):
        _,D_enc = chol_LD_Crypten(crypt_PSD_mat)
        D_log_enc = D_enc.log()
        return D_log_enc.sum()
        

In [66]:
a = torch.randn((3,3))
A = a.T @ a + torch.eye(3)
A_enc = crypten.cryptensor(A)

In [67]:
L_enc, D_enc = chol_LD_Crypten(A_enc) 
L_enc.get_plain_text(), D_enc.get_plain_text()

(tensor([[ 1.0000,  0.0000,  0.0000],
         [-0.7799,  1.0000,  0.0000],
         [-0.0382, -0.0490,  1.0000]]),
 tensor([2.4639, 2.1603, 1.4012]))

In [68]:
LD = torch.linalg.ldl_factor(A).LD
D = torch.diag(LD)
D

tensor([2.4639, 2.1603, 1.4011])

In [69]:
torch.slogdet(A)

torch.return_types.slogdet(
sign=tensor(1.),
logabsdet=tensor(2.0093))

In [70]:
logdet_Crypten(A_enc).get_plain_text()

tensor(2.0081)

Implementing linear EIG calc in crypten

In [195]:
n_host_sample = 2000 
sigma_error = 1
d = 5
A = torch.randn((d,d))
A = 1/(torch.det(A)) * A

T_allocation_host = torch.randn(d)
T_allocation_host = 100/torch.norm(T_allocation_host)*T_allocation_host

mu_nc = torch.randn(d)
mu_nc = 1/torch.norm(mu_nc)*mu_nc

mu_c = torch.randn(d)
mu_c = 1/torch.norm(mu_c)*mu_c

mu = torch.concat([mu_nc,mu_c])

In [196]:
X_host_no_T = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host = torch.concat([X_host_no_T,X_host_times_T],dim=1)

Y_host = X_host @ mu
Y_host = (1/Y_host.std()) * (Y_host-Y_host.mean()) + sigma_error * torch.randn_like(Y_host)

In [197]:
X_host_no_T = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host = torch.concat([X_host_no_T,X_host_times_T],dim=1)

In [198]:
sigma_error = 1
prior_mean = torch.zeros(2 * d)
beta_0, sigma_0_sq,inv_cov_0 = prior_mean, sigma_error,torch.eye(2*d)

In [208]:
X_host_no_T_2 = (torch.randn((n_host_sample,d)) @ A ) 
T_host_2 = torch.bernoulli(torch.sigmoid(X_host_no_T@ T_allocation_host))
X_host_times_T_2 = (T_host.unsqueeze(dim=0).T * X_host_no_T)
X_host_2 = torch.concat([X_host_no_T,X_host_times_T],dim=1)

In [209]:
torch.logdet((X_host.T @ X_host) @ torch.linalg.inv((X_host_2.T @ X_host_2)) + torch.eye(2*d))

tensor(6.9315)

In [210]:
B = crypten.mpc.MPCTensor(((X_host.T @ X_host)))
C = crypten.mpc.MPCTensor(torch.linalg.inv((X_host_2.T @ X_host_2)))
logdet_Crypten(B @ C + torch.eye(2*d)).get_plain_text()

tensor(6.5459)

In [211]:
torch.linalg.ldl_factor((X_host.T @ X_host) @ torch.linalg.inv((X_host.T @ X_host)))

torch.return_types.linalg_ldl_factor(
LD=tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-5.7221e-06,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.8147e-06,  8.2013e-05,  9.9996e-01,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.4305e-06, -8.1851e-12,  1.7167e-05,  1.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-7.6294e-06,  1.5258e-05,  7.6284e-06, -1.5259e-05,  1.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 2.8610e-06,  3.8146e-06, -3.8152e-06, -2.2888e-05,  1.1444e-05,
          9.9999e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.8147e-06,  2.6702e-05,  1.3350e-05, -3.0518e-05,  1.9

In [202]:
(X_host.T @ X_host)

tensor([[  7899.1099,   8018.7383,   2116.7344,   -763.3619,  -8738.0908,
           4323.2778,   4352.1602,   1029.1210,   -378.6817,  -4779.8799],
        [  8018.7383,  42040.4297,  38230.3984,   2255.5239, -10139.5957,
           4352.1602,  21202.4531,  18866.3945,   1336.1771,  -5086.8818],
        [  2116.7344,  38230.3984,  56501.1328,  -4394.6880,  -5863.3267,
           1029.1210,  18866.3945,  28028.5078,  -2251.7041,  -2944.0967],
        [  -763.3619,   2255.5239,  -4394.6880,   6783.1279,   7662.6147,
           -378.6817,   1336.1771,  -2251.7041,   3629.0737,   4036.5630],
        [ -8738.0908, -10139.5957,  -5863.3267,   7662.6147,  21135.7070,
          -4779.8799,  -5086.8818,  -2944.0967,   4036.5630,  11025.7285],
        [  4323.2778,   4352.1602,   1029.1210,   -378.6817,  -4779.8799,
           4323.2778,   4352.1602,   1029.1210,   -378.6817,  -4779.8799],
        [  4352.1602,  21202.4531,  18866.3945,   1336.1771,  -5086.8818,
           4352.1602,  21202.453

In [203]:
B.get_plain_text()

tensor([[  7899.1099,   8018.7383,   2116.7344,   -763.3619,  -8738.0908,
           4323.2778,   4352.1602,   1029.1210,   -378.6817,  -4779.8799],
        [  8018.7383,  42040.4297,  38230.3984,   2255.5239, -10139.5957,
           4352.1602,  21202.4531,  18866.3945,   1336.1771,  -5086.8818],
        [  2116.7344,  38230.3984,  56501.1328,  -4394.6880,  -5863.3267,
           1029.1210,  18866.3945,  28028.5078,  -2251.7041,  -2944.0967],
        [  -763.3619,   2255.5239,  -4394.6880,   6783.1279,   7662.6147,
           -378.6817,   1336.1771,  -2251.7041,   3629.0737,   4036.5630],
        [ -8738.0908, -10139.5957,  -5863.3267,   7662.6147,  21135.7070,
          -4779.8799,  -5086.8818,  -2944.0967,   4036.5630,  11025.7285],
        [  4323.2778,   4352.1602,   1029.1210,   -378.6817,  -4779.8799,
           4323.2778,   4352.1602,   1029.1210,   -378.6817,  -4779.8799],
        [  4352.1602,  21202.4531,  18866.3945,   1336.1771,  -5086.8818,
           4352.1602,  21202.453

In [204]:
(B + B + torch.eye(2*d)).get_plain_text()

tensor([[ 15799.2197,  16037.4766,   4233.4688,  -1526.7239, -17476.1816,
           8646.5557,   8704.3203,   2058.2419,   -757.3634,  -9559.7598],
        [ 16037.4766,  84081.8594,  76460.7969,   4511.0479, -20279.1914,
           8704.3203,  42404.9062,  37732.7891,   2672.3542, -10173.7637],
        [  4233.4688,  76460.7969, 113003.2656,  -8789.3760, -11726.6533,
           2058.2419,  37732.7891,  56057.0156,  -4503.4082,  -5888.1934],
        [ -1526.7239,   4511.0479,  -8789.3760,  13567.2559,  15325.2295,
           -757.3634,   2672.3542,  -4503.4082,   7258.1475,   8073.1260],
        [-17476.1816, -20279.1914, -11726.6533,  15325.2295,  42272.4141,
          -9559.7598, -10173.7637,  -5888.1934,   8073.1260,  22051.4570],
        [  8646.5557,   8704.3203,   2058.2419,   -757.3634,  -9559.7598,
           8647.5557,   8704.3203,   2058.2419,   -757.3634,  -9559.7598],
        [  8704.3203,  42404.9062,  37732.7891,   2672.3542, -10173.7637,
           8704.3203,  42405.906

In [205]:
L,D = chol_LD_Crypten((B + B + torch.eye(2*d)))
L.get_plain_text(),D.get_plain_text()

(tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 1.4815e+09,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-1.1515e+09, -1.8658e+09,  1.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 1.1783e+09, -9.7138e+08, -9.3201e+08,  1.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-1.6907e+09, -2.3217e+08,  2.0131e+09, -1.9774e+09,  1.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-1.5457e+09,  1.0809e+09,  8.7649e+08, -8.4053e+08, -5.5151e+08,
           1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-1.7891e+09,  2.0728e+09, -1.3313e+09,  1.2855e+09, -1.3245e+09,
           2.3197e+

In [206]:
logdet_Crypten(B + B + torch.eye(2*d)).get_plain_text()

tensor(-6.1170e+10)